In [20]:
import os
import json
import pandas as pd 
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import FeatureHasher
from sklearn.metrics import mean_absolute_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
import pickle
import joblib 

In [2]:
os.chdir('..')

In [3]:
np.random.seed(0)

In [4]:
test = pd.read_csv('ENG_DATA/SELECTED/12-10_all_test.csv', index_col = [0])
train = pd.read_csv('ENG_DATA/SELECTED/12-10_all_train.csv', index_col = [0])

In [5]:
test.drop(columns = ['orig_list_price', 'full_address'], inplace=True)
train.drop(columns = ['orig_list_price', 'full_address'], inplace=True)

In [ ]:
def get_prediction_error(model, X, y, X_val, y_val):
    model.fit(X, y)
    y_pred = model.predict(X_val)
    
    return score

In [21]:
def train_model_with_location(test, train):
    numerical_features = list(train.drop(columns = ["sale_price"]).select_dtypes(include = 'number').columns)
    categorical_features = list(train.drop(columns = ["sale_price"]).select_dtypes(exclude = 'number').columns)
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', FeatureHasher(), categorical_features)])
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', RandomForestRegressor(n_estimators=50, max_depth=20,random_state=0))])
    x_test = test.drop(columns = ['sale_price'])
    y_test = test["sale_price"]
    x_train = train.drop(columns = ['sale_price'])
    y_train = train["sale_price"]
    clf.fit(x_train, y_train)
    print("model score: %.3f" % clf.score(x_test, y_test))
    return clf.predict(x_test)

In [22]:
y_pred_loc = train_model_with_location(test, train)

AttributeError: 'str' object has no attribute 'items'

In [ ]:
# Source for Feature Mixing and Pipelining - current problem it's too slow with One Hot Encoding
# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html#sphx-glr-auto-examples-compose-plot-column-transformer-mixed-types-py

In [ ]:
# numerical_features = list(train.drop(columns = ['sale_price', 'latitude', 'longitude', 'elevation']).select_dtypes(include = 'number').columns)
# categorical_features = list(train.drop(columns = ['sale_price', 'latitude', 'longitude', 'elevation']).select_dtypes(exclude = 'number').columns)

KeyboardInterrupt: 

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
r2_score(y_pred, y_test)

In [ ]:
mean_absolute_error(y_pred, y_test) / np.mean(y_test)

In [ ]:
median_absolute_error(y_pred, y_test) / np.mean(y_test)

In [ ]:
np.mean(np.abs((y_pred - y_test) / y_test)) * 100

In [ ]:
joblib.dump(clf_f, 'ENG_DATA/PICKLES/bubble4-3-all.pickle' + '.z')  

In [23]:
#combine data-set predictions 
clf_hedon = joblib.load('ENG_DATA/PICKLES/bubble4-3.pickle' + '.z') 
y_pred_hedon = clf_hedon.predict(x_test.drop(columns = ['latitude', 'longitude', 'elevation']))

ValueError: could not convert string to float: 'San Francisco'

In [ ]:
x_test["pred_all"] = y_pred
x_test["pred_hedon"] = y_pred_hedon

In [2]:
import matplotlib.pyplot as plt
from geopy import distance
from shapely.geometry import Point
import geopandas
import geocoder

In [5]:
# import sf 
sf = geopandas.read_file('RAW_DATA/sf.geojson')
sf.crs = {'init': 'epsg:4326'}
sf = sf.rename(columns={'geometry': 'geometry','nhood':'neighborhood_name'}).set_geometry('geometry')

In [ ]:
df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
geo_df = geopandas.GeoDataFrame(df, geometry='geometry')
geo_df.crs = {'init': 'epsg:4326'}

In [ ]:
# fig, ax = plt.subplots(1, figsize=(13,10))
fig, ax = plt.subplots(1, figsize=(13,10))
sf_map = sf.plot(ax=ax, color='white', edgecolor='black')
geo_crime.plot(ax=sf_map, marker="o", markersize=8, alpha=0.3)
ax.set_title("San Francisco Bubble Mapping")
plt.show()